# Summary

Do I sleep less on nights when the Warriors play?

In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import nba_py

sns.set_context('poster')
import plotly.offline as py
import plotly.graph_objs as go

In [3]:
py.init_notebook_mode(connected=True)

In [4]:
data_path = os.path.join(os.getcwd(), os.pardir, 'data', 'interim', 'sleep_data.csv')
df_sleep = pd.read_csv(data_path, index_col='shifted_datetime', parse_dates=True)

In [5]:
df_sleep.index += pd.Timedelta(hours=12)

In [6]:
sleep_day = df_sleep.resample('1D').sum().fillna(0)

In [7]:
from nba_py import league

In [8]:
gswlog = league.GameLog(player_or_team='T')

In [9]:
league_logs = gswlog.json['resultSets'][0]['rowSet']

In [10]:
columns = gswlog.json['resultSets'][0]['headers']

In [11]:
df_league = pd.DataFrame(league_logs, columns=columns)

In [12]:
df_league.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')

In [13]:
gsw_games = df_league[df_league['TEAM_ABBREVIATION'] == 'GSW']

In [14]:
len(gsw_games)

82

In [15]:
gsw_games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22016,1610612744,GSW,Golden State Warriors,0021600223,2016-11-23,GSW vs. LAL,W,240,53,...,42,48,47,7,5,10,20,149,43,1
1,22016,1610612744,GSW,Golden State Warriors,0021600707,2017-01-28,GSW vs. LAC,W,240,57,...,39,48,35,9,7,11,17,144,46,1
5,22016,1610612744,GSW,Golden State Warriors,0021600314,2016-12-05,GSW vs. IND,W,240,54,...,38,54,45,12,13,11,22,142,36,1
10,22016,1610612744,GSW,Golden State Warriors,0021601152,2017-04-02,GSW vs. WAS,W,240,50,...,34,44,34,7,10,16,20,139,24,1
13,22016,1610612744,GSW,Golden State Warriors,0021600299,2016-12-03,GSW vs. PHX,W,240,49,...,32,38,37,9,5,17,19,138,29,1


In [16]:
gsw_dates = gsw_games['GAME_DATE']

In [46]:
toplot = sleep_day['minutesAsleep']/60.

data = []

data.append(
    go.Scatter(
        x=toplot.index,
        y=toplot.values,
        name='Hours Asleep'
    )
)

shapes = []
for idate, gsw_date in enumerate(gsw_dates):
    if idate == 0:
        showlegend = True
    else:
        showlegend = False
    trace0 = go.Scatter(
        x=[gsw_date],
        y=[toplot.dropna().min()],
        mode='markers',
        name='Golden State Warriors Game',
        marker=dict(
            color='salmon'
        ),
        showlegend=showlegend
    )
    data.append(trace0)

layout = go.Layout(
    title="Daily Sleep Total, 6pm to 6pm",
    yaxis=dict(
        title='Hours Asleep'
    ),
)

fig = {
    'data': data,
    'layout': layout,
}

py.iplot(fig, filename='DailySleepTotal_GSWGames')

The NBA season starts at the end of October. I got my fitbit near the beginning of November, so there is a lot of overlap.

A simple test: sleep on the night of a Warriors game vs. all other nights. 

> Here, we have to be careful about the definition of a night of sleep. I follow Fitbit's convention and assert that hours of sleep on a given date correspond to falling asleep the night before and waking up the day of. So if I fall asleep on Tuesday August 8th at 11pm and wake up on Wednesday August 9th at 7am, then I got 8 hours of sleep on August 9th.

To answer my question about sleep on the night of a Warriors game, I need to compare sleep the day AFTER the Warriors game to all other nights. E.g., the Warriors had a game on April 8, 2017. The relevant night of sleep for that game is April 9, 2017. A quick and dirty way to tackle this is by adding 24 hours to the Warriors game dates.

In [17]:
gsw_dates = pd.to_datetime(gsw_dates)

In [18]:
gswdatedf = pd.DataFrame(index=gsw_dates)

In [19]:
gswdatedf['game_status'] = 1

In [20]:
gswdatedf = gswdatedf.resample('1D').sum().fillna(0)

In [21]:
gswdatedf_next = gswdatedf.copy()
gswdatedf_next.index += pd.Timedelta(hours=24)

In [22]:
sleepgsw = sleep_day.join(gswdatedf_next, how='inner')

In [23]:
sleepgswyes = sleepgsw.groupby('game_status').mean()

In [24]:
sleepgswyes['minutesAsleep'] / 60

game_status
0.0    6.949588
1.0    6.490000
Name: minutesAsleep, dtype: float64

Interesting! The night of a Warriors game, I get almost half an hour less sleep than other nights. This is still not an enormous difference, so we should be cautious in our interpretation. In particular, there could be other factors at work driving this difference. For now, let's keep this in mind as we continue to explore possible predictors of my sleep patterns.